In [1]:
import random
random.seed(8)

import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D

from jim import Vector, Body, Universe, transpose_states, implement_universal_force, coulomb_force

Vector(-5.0) Vector(2.5)
Vector(-10.0) Vector(5.0)
[(1, 3), (2, 4)]


In [2]:
number_of_bodies = 5

In [3]:
def kinetic_energy(states):
    energy = 0
    for state in states:
        energy += 0.5 * state.m * state.v.norm ** 2
    return energy

In [4]:
def random_vector(a):
    return Vector(*[random.uniform(-a, a) for _ in range(3)])

In [5]:
bodies = [
    Body(random_vector(5), random_vector(2), None, 1)
    for _ in range(number_of_bodies)
]

for body in bodies:
    body.q = 0.0002

In [6]:
universe = Universe(Vector(0, 0, 0))
for body in bodies:
    universe.add(body)
implement_universal_force(universe, coulomb_force)

`background_force` ist die Kraft, die durch das Potential 
$$ U(\vec r) = \frac{1}{2}m\omega^2r^2 $$
hervorgerufen wird:
$$ \vec{F_b}(\vec r) = - \nabla U(\vec r) = -m \omega^2 \vec r $$
Der genaue Wert von $\omega$ ist nur ein Skalierungsfaktor und hier nicht von Interesse.

In [7]:
ω = 1
def potential(body):
    return 0.5 * body.m * ω ** 2 * body.r.norm ** 2

In [8]:
def background_force(body):
    return -body.m * ω ** 2 * body.r

Die Reibungskraft ist ein wenig willkürlich, hat sich aber in einer Billardsimulation (wo sie als Rollreibung verwendet wird) als recht realistisch wirkend herausgestellt. Der Optik wegen wird sie unten noch mit ½ skaliert.

In [9]:
def friction(body):
    speed = body.v.norm
    if speed < 0.7:
        return -body.v / 0.7
    elif speed < 1:
        return -body.v.unit
    else:
        return -body.v

In [10]:
for body in universe.objects:
    body.add_force(background_force)
    body.add_force(lambda body: friction(body) / 2)

----

In [11]:
dt = 1. / 30 # 30fps
frames = 900

In [12]:
# set up figure and animation
fig = plt.figure()
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
ax = fig.add_subplot(
    111,
    aspect='equal',
    autoscale_on=False,
    projection="3d",
)
ax.grid()
ax.set_xlim3d(-7, 7)
ax.set_ylim3d(-7, 7)
ax.set_zlim3d(-7, 7)

(-7, 7)

Ein roter Punkt im Ursrung zur Orientierung:

In [13]:
ax.scatter([0], [0], c="red");

In [14]:
# particles holds the locations of the particles
particles, = ax.plot([], [], 'bo', ms=20)

In [15]:
def init():
    """initialize animation"""
    particles.set_data([], [])
    particles.set_3d_properties([])
    return particles,

In [16]:
def exhaust(iterator):
    for _ in iterator:
        pass

In [17]:
def animate(i):
    """perform animation step"""
    exhaust(universe.simulate(dt, 0.005))
    states = universe.state()

    transposed = transpose_states(states)
    particles.set_data(*transposed[:2])
    particles.set_3d_properties(transposed[2])
    ax.view_init(i * 360 / 900 - 90, i)
    return particles,

In [18]:
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=frames,
    interval=10,
    blit=True,
    init_func=init
)

In [19]:
# save the animation as an mp4.
ani.save('{}_body_3d_animation.mp4'.format(number_of_bodies), fps=30, extra_args=['-vcodec', 'libx264'])

In [20]:
# Turn off axes because they flicker.
plt.axis("off")
# Uncomment to see a live and interactive view of the system.
#plt.show()

(-7.0, 7.0, -7.0, 7.0)

In [21]:
print(universe)

Universe(time=26.999999999998913, objects=[BodyState(r=Vector(-2.251578040779315, 6.711909523779487, -2.990735488985451), v=Vector(-0.0009439082877340572, 0.00259613215031427, -0.0015859046093530018), a=Vector(0.0008622984186741382, -0.004475354359809012, 0.0025808857443740205), m=1), BodyState(r=Vector(6.494443743412582, 0.20741975853468717, -4.400496105459135), v=Vector(-0.002296096161745937, 0.006764195895146839, -0.0034092049683208665), a=Vector(-0.0005400567478470052, -0.0037429345459582767, 0.00257690181619533), m=1), BodyState(r=Vector(-2.0710188076000158, -6.5872624695977375, -3.37532837185105), v=Vector(0.005692100035242434, -0.0066267700197925545, 0.007096054375399577), a=Vector(-0.0022522205730200577, 0.005652530967120948, -0.0020110965582340783), m=1), BodyState(r=Vector(4.317118765857014, -0.10608172975418122, 6.356990522097974), v=Vector(0.0008219685798570731, -0.011540231399374991, 0.0011989028050460484), a=Vector(-0.0022359634674040626, 0.005541428200363267, -0.00150777

In [22]:
for i in range(number_of_bodies):
    for j in range(i + 1, number_of_bodies):
        print(i, j, (universe.objects[i].r - universe.objects[j].r).norm)

0 1 10.9903916741622
0 2 13.305956900263368
0 3 13.304614389341074
0 4 10.992027921957163
1 2 10.981157688337282
1 3 10.980097712821902
1 4 15.69665136916328
2 3 13.324114848403616
2 4 10.981865544449503
3 4 10.98232058920327


In [23]:
for body in universe.objects:
    print(body.r.norm)

7.685299727024213
7.847623109371551
7.685960406001747
7.6850566861894976
7.849039852472535


In [24]:
sum((body.r for body in universe.objects), Vector.null_vector(3))

Vector(-0.0013221812546708733, 0.0016087533137151833, -0.001253938871262683)